In [ ]:
"""
Todos
- [ ] Update requirements.txt
- [ ] Test anaconda

Questions

- [?] Can we install pywin32?
- [?] Where is our item name?
- [ ] What's the cutoff date? Just in the past? Older than x months?
- [ ] What's the subject line?
"""

In [1]:
# Important: To get win32 working, we need to do the following steps
# Using pip:
# pip install pypiwin32
# Then open an administrator shell and do
# venv\Scripts\activate
# python venv\Scripts\pywin32_postinstall.py -install

import win32com.client as win32
import os
import psutil
import subprocess
import pandas as pd

from dataclasses import dataclass
from template import MAIL_SUBJECT, MAIL_BODY

In [2]:
# Read input file
EXCEL_INPUT_FILE = "EC_SDS_TESTFILE.xlsx"

excel_workbook = pd.read_excel(EXCEL_INPUT_FILE)
excel_workbook

# TODO: Filter by `last_update` (last_update needs to be older than x)
# 
# TODO: 

,art,language,supplier,last_update,vendor_master.salutation,vendor_master.email
0,RX072299,EN,ADEKA CORPORATION（昭和興産株式会社）,2019-01-28,昭和興産株式会社,kazuhiro.kamehameha@nagase.co.jp
1,RX084154,EN,DAICEL CORPORATION（森六ケミカルズ株式会社）,2019-04-15,森六ケミカルズ株式会社,igor.susak@basf.com
2,RX084170,EN,DAICEL CORPORATION（森六ケミカルズ株式会社）,2019-04-15,森六ケミカルズ株式会社,igor.susak@basf.com


In [12]:
# Set environment variables
OUTLOOK_APP_PATH = '"C:/Program Files/Microsoft Office/root/Office16/OUTLOOK.EXE"'
OUTLOOK_APP_NAME = 'OUTLOOK.EXE'

# Email dataclass
@dataclass
class Email:
    sendTo: str
    subject: str
    body: str

# Item dataclass
@dataclass
class Item:
    modifiedDate: str  # TODO: Change to date object
    itemNo: str
    itemName: str
    supplierName: str
    
    def createMessage(self) -> str:
#         return f"""
#         Dear {self.supplierName}
        
#         Please send us new data for {self.itemName} [#{self.itemNo}],
#         which has expired since {self.modifiedDate}.
#         """
    
        return f"""お取引先様各位

いつもお世話になっております。

現在弊社システムの定期的なメンテナンスを行っており、最新版のSDSを集めております。
つきましては貴社より購入しております、下記製品の和文と英文の両方のSDSの御提供をお願い致します。
{self.itemName} [#{self.itemNo}]

なお、以前に頂戴したSDSが弊社のシステムにあり記載されている改訂日が下記の通りです。
{self.modifiedDate}

改訂されていない場合はその旨お知らせいただければ幸いでございます。
お忙しいところ恐れ入りますが、ご協力のほど何卒よろしくお願い申し上げます。
"""

In [13]:
def open_outlook(path: str) -> None:
    """ Tries to open Outlook App from given path
    """
    try:
        subprocess.Popen(OUTLOOK_APP_PATH)
    except FileNotFoundError as e:
        print(f"Error: File not found at {OUTLOOK_APP_PATH}")

def app_is_running(app_name: str) -> bool:
    """ Checks running processes to see if a given app is running.
    Returns:
        True -- if app is running else False
    """
    # Get names for all running processes
    running_processes = (psutil.Process(pid).name() for pid in psutil.pids())
        
    for process in running_processes:
        if app_name in process:
            return True

    return False

def create_draft(mail: Email) -> None:
    """Takes an Email dataclass and saves a draft email.
    """
    outlook = win32.Dispatch('outlook.application')
    draft = outlook.CreateItem(0)
    draft.To = mail.sendTo
    draft.Subject = mail.subject
    draft.body = mail.body
    draft.save()


In [14]:
# Make sure we've got an email client running
if not app_is_running(OUTLOOK_APP_NAME):
    print("App not running, opening now.")
    open_outlook(OUTLOOK_APP_PATH)
else:
    print(f"{OUTLOOK_APP_NAME} is already running.")

OUTLOOK.EXE is already running.


In [15]:
# Create a test item
test_item = Item(
    '2019/04/14',
    '1234',
    'Ice Cream',
    'Mr. Freeze',
)

# Create a test email
test_mail = Email(
    'test@example.com',
    'Dataclass Email Test',
    test_item.createMessage(),
)

In [16]:
create_draft(test_mail)